In [18]:
import fisher 
import foregrounds_fisher as fg
import spectral_distortions as sd
import numpy as np
from noise_functions import *

# Example using original version of sd_foregrounds (i.e. example.py)

In [ ]:
# PIXIE with all signals is the default. 
fish = fisher.FisherEstimation()
# args are stored in fish.args, values are stored in dictionary fish.argvals, 
# and fisher uncertainties are in fish.errors
fish.run_fisher_calculation()
# print the errors in sigma
fish.print_errors()

# To set the signals by hand, just modify the fncs arg here:
fish = fisher.FisherEstimation()
fish.set_signals(fncs=[sd.DeltaI_reltSZ_2param_yweight, sd.DeltaI_DeltaT, sd.DeltaI_mu,
                       fg.thermal_dust_rad, fg.cib_rad, fg.jens_freefree_rad,
                       fg.jens_synch_rad, fg.spinning_dust, fg.co_rad])
fish.run_fisher_calculation()
fish.print_errors()


# To change the frequencies (Hz), duration (months), or scale the noise by mult, 
# turn off the step function bandpass or change fsky, 
# edit any of the following
fish = fisher.FisherEstimation(fmin=5.e9, fmax=1.e12, fstep=5.e9, duration=60, mult=0.1, bandpass=False, fsky=0.5)


# Lastly to put priors (in fractions of the parameter value), drop the first n bins or mask out Galactic CO lines do:
fish = fisher.FisherEstimation(priors={'Td':0.1, 'Asd':0.01}, drop=2, doCO=True) 


# Example computing fisher forecast for a new instrument set-up

## Simple example computing for a given set of bands & detector counts

In [20]:
# define some bands and detectors for each band 
optimized_data=np.loadtxt("data/mu_1to2000_width1_snr25_bolo10_optimized_bands.txt", delimiter=',')
BANDS=optimized_data[:,:2].reshape(len(optimized_data[:,0]),2)
print('Frequency Bands [Hz]:')
print(BANDS)
DETS=np.array([  2.,   4.,   6.,   6.,   8.,  20.,  100., 100., 100., 100.,100,100,100, 100., 100., 100.])

print('Detector Counts:')
print(dets)


Frequency Bands [Hz]:
[[1.000e+00 2.000e+00]
 [2.000e+00 3.000e+00]
 [3.000e+00 4.000e+00]
 [4.000e+00 1.000e+01]
 [1.000e+01 1.300e+01]
 [1.300e+01 4.300e+01]
 [4.300e+01 9.700e+01]
 [9.700e+01 1.720e+02]
 [1.720e+02 3.130e+02]
 [3.130e+02 5.320e+02]
 [5.320e+02 7.570e+02]
 [7.570e+02 9.130e+02]
 [9.130e+02 1.174e+03]
 [1.174e+03 1.501e+03]
 [1.501e+03 1.858e+03]
 [1.858e+03 2.000e+03]]
Detector Counts:
[  2.   4.   6.   6.   8.  20. 100. 100. 100. 100. 100. 100. 100. 100.
 100. 100.]


In [29]:
%%time 
#run fisher
FSKY=0.7 
DUR_MONTHS=12
HEMT_AMPS=True
HEMT_FREQ=10
FILE_PREFIX="SPECTER"

fish = fisher.FisherEstimation(fsky=FSKY, #observed sky fraction/percentage of TOD
                               duration=DUR_MONTHS, #observation duration in months
                               bandpass=False, #assume center frequencies are just the midpoint of the band edges 
                               instrument='new_instrument', # select new_instrument (default is 'pixie')
                               file_prefix=FILE_PREFIX, # prefix for all the bolocalc output files
                               freq_bands=BANDS, #bands array (shape is (# of bands, 2))
                               Ndet_arr=DETS, #detector array (1-D array of length # of bands )
                               hemt_amps=HEMT_AMPS, #use hemt amplifiers for low frequencies (recommended for anything below 10GHz)
                               hemt_freq=HEMT_FREQ, #below which frequency to use hempt amplifiers if hemt_amps=True
                               noisefile=None, #use instantanous sensitivity file in muK-rt(s) or compute sensitity using bolocalc
                               priors={}, #priors on model parameters
                               systematic_error=[], #compute bias for a given systematic error
                               arg_dict={}) #change fiducial parameters
fish.run_fisher_calculation()
fish.print_errors()

No new bands found, simulating all
clearing bands

Simulting 1 experiment realizations each with 1 detector realizations and 1 sky realizations. Total sims = 1

[                                                                                                    ] 0.0%
[====================================================================================================] 100.0%

saving sensitivities to /Users/asabyr/Documents/software/bolocalc-space/Experiments/specter_v1/SPECTERsens_out.npy
mu_amp 5.038368917461055
y_tot 955.4305529683442
kT_yweight 33.4362894814056
DeltaT_amp 37156.56509210914
Ad 799.1616093925419
Bd 3317.6696531275315
Td 51770.52699568453
Acib 221.75802836835746
Bcib 720.6981154138261
Tcib 2111.460574164406
EM 1277.4214382240891
As 1816.5367169901049
alps -5654.802353574107
w2s 575.7349158517261
Asd 8016.479490669191
Aco 2092.7838419296304
CPU times: user 988 ms, sys: 56.2 ms, total: 1.04 s
Wall time: 15.6 s


## Precompute instantanous noise to avoid running bolocalc each time


In [25]:
print("Precomputing instantanous noise")
specter_freqs, specter_noise=getnoise_raw(path="data/", 
                                          bands=BANDS, 
                                          prefix=FILE_PREFIX,
                                          hemt_amps=HEMT_AMPS,
                                          hemt_freq=HEMT_FREQ)
print(f"Saved {FILE_PREFIX}_raw_sensitivity.txt file in /data")

Precomputing instantanous noise
No new bands found, simulating all
clearing bands

Simulting 1 experiment realizations each with 1 detector realizations and 1 sky realizations. Total sims = 1

[                                                                                                    ] 0.0%
[====================================================================================================] 100.0%

saving sensitivities to /Users/asabyr/Documents/software/bolocalc-space/Experiments/specter_v1/SPECTERsens_out.npy
Saved SPECTER_raw_sensitivity.txt file in /data


In [27]:
%%time
print("Now we can compute fisher super fast")

fish = fisher.FisherEstimation(fsky=FSKY, #observed sky fraction/percentage of TOD
                               duration=DUR_MONTHS, #observation duration in months
                               bandpass=False, #assume center frequencies are just the midpoint of the band edges 
                               instrument='new_instrument', # select new_instrument (default is 'pixie')
                               file_prefix=FILE_PREFIX, # prefix for all the bolocalc output files
                               freq_bands=BANDS, #bands array (shape is (# of bands, 2))
                               Ndet_arr=DETS, #detector array (1-D array of length # of bands )
                               hemt_amps=HEMT_AMPS, #use hemt amplifiers for low frequencies (recommended for anything below 10GHz)
                               hemt_freq=HEMT_FREQ, #below which frequency to use hempt amplifiers if hemt_amps=True
                               noisefile="data/SPECTER_raw_sensitivity.txt", #use instantanous sensitivity file in muK-rt(s) or compute sensitity using bolocalc
                               priors={}, #priors on model parameters
                               systematic_error=[], #compute bias for a given systematic error
                               arg_dict={}) #change fiducial parameters
fish.run_fisher_calculation()
fish.print_errors()

Now we can compute fisher super fast
mu_amp 5.038368917461055
y_tot 955.4305529683442
kT_yweight 33.4362894814056
DeltaT_amp 37156.56509210914
Ad 799.1616093925419
Bd 3317.6696531275315
Td 51770.52699568453
Acib 221.75802836835746
Bcib 720.6981154138261
Tcib 2111.460574164406
EM 1277.4214382240891
As 1816.5367169901049
alps -5654.802353574107
w2s 575.7349158517261
Asd 8016.479490669191
Aco 2092.7838419296304
CPU times: user 131 ms, sys: 6.97 ms, total: 138 ms
Wall time: 131 ms


# Example computing bias 


# Example computing fisher errors using different fiducial values for the model